In [267]:
import random
import sqlite3

sqlite_file = 'abc_code.db'
table_name = 'fake_scores'

conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

column1 = 'question_id'
question_id = 'question-1'
column2 = 'score'
limit = 852

data = list()
target = list()

for points in (0, 1, 2, 3):
    c.execute('SELECT answered, score FROM {tn} WHERE {cn1}="{val1}" AND {cn2}="{val2}" LIMIT {limit}'.\
            format(tn=table_name, cn1=column1, val1=question_id, cn2=column2, val2=points, limit=limit))
    rows = c.fetchall()

    data.extend([x[0] for x in rows])
    target.extend([x[1] for x in rows])

print(data[0])
print(random.sample(target, 10))

conn.close()

that provides A no An
[3, 2, 2, 0, 3, 1, 2, 2, 0, 1]


In [251]:
import sys
sys.path.append('/Library/Python/2.7/site-packages')
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# MultinomialNB()
# 0.40
from sklearn.linear_model import SGDClassifier
# SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
#0.20 0.40
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
#0.599999, 0.20, 0.40
text_clf = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(2, 2), min_df=1)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)),
])

In [252]:
text_clf = text_clf.fit(data, target)

In [255]:
test_data = ['if then', 'yes no', 'instructions depend', 'case depend', 'not related']
test_target = [3, 3, 3, 3, 0]
predicted = text_clf.predict(test_data)
print(predicted)

[2 3 2 2 0]


In [256]:
import numpy as np
np.mean(predicted == test_target)  

0.40000000000000002

In [265]:
from __future__ import division
def score_prediction(predicted, target, target_names):
    possible = len(target) * len(target_names)
    score = possible
    for p, t in zip(predicted, target):
        diff = abs(t - p)
        score -= diff
    return score / possible

In [266]:
score_prediction(predicted, test_target, ["criteria-4", "criteria-3", "criteria-2", "criteria-1"])

0.84999999999999998

In [241]:
from sklearn import metrics
print(metrics.classification_report(test_target, predicted,
    target_names=["criteria-4", "criteria-3", "criteria-2", "criteria-1"]))

             precision    recall  f1-score   support

 criteria-4       1.00      1.00      1.00         1
 criteria-3       0.00      0.00      0.00         0
 criteria-2       1.00      0.25      0.40         4

avg / total       1.00      0.40      0.52         5



In [242]:
metrics.confusion_matrix(test_target, predicted)

array([[1, 0, 0],
       [0, 0, 0],
       [0, 3, 1]])